# CCYOE Analytics - Getting Started

This notebook provides a quick introduction to the CCYOE Analytics package.

## What is CCYOE?

The Cross-Collateral Yield Optimization Engine (CCYOE) is Cambi Protocol's innovative system that:
- **Unifies Yield Distribution**: Treats excess yields from high-performing assets as a shared resource
- **Optimizes Cross-Asset Returns**: Redistributes excess yield to boost under-performing assets
- **Creates Network Effects**: Success in one asset benefits all users through intelligent rebalancing
- **Maximizes Capital Efficiency**: Ensures no yield goes unutilized while maintaining risk isolation

## Traditional vs CCYOE Approach

**Traditional protocols:**
- USDC earns T-bill yields (5%) - Circle keeps the difference
- BTC earns DeFi lending (3%) - high risk for low returns  
- Local currencies earn local rates - usually locked and controlled

**Cambi's CCYOE:**
- cmBRL generates 25% yield (5% excess above 20% target)
- Excess 5% gets redistributed: 40% to under-supplied assets, 30% to strategic growth, 20% proportionally, 10% to treasury
- Result: cmUSD achieves 14-18% vs competitors' 5%, cmBTC gets 5-8% vs competitors' 3%

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import CCYOE Analytics
import sys
sys.path.append('..')

from cambi_analytics import (
    DataLoader, YieldAnalyzer, 
    CCYOEBacktester, OptimizationConfig,
    get_config
)

print("✅ CCYOE Analytics imported successfully!")

## Quick Example: Load Sample Data

In [ ]:
# Load sample Brazilian market data
data_loader = DataLoader()

data = data_loader.load_sample_data(
    data_type='brazilian_market',
    start_date='2023-01-01',
    end_date='2024-01-01'
)

print(f"📊 Loaded {len(data)} days of data")
print(f"💱 Assets: {[col for col in data.columns if col != 'date']}")

# Show sample data
data.head()

## Quick Analysis: Asset Yields

In [ ]:
# Analyze yields
analyzer = YieldAnalyzer(data, assets=['cmBTC', 'cmUSD', 'cmBRL'])

# Calculate average yields
print("💰 Average Yields:")
for asset in ['cmBTC', 'cmUSD', 'cmBRL']:
    avg_yield = analyzer.get_average_yield(asset)
    print(f"   {asset}: {avg_yield:.0f} bp ({avg_yield/100:.1f}%)")

# Plot yield trends
plt.figure(figsize=(12, 6))
plt.plot(data['date'], data['cmBRL']/100, label='cmBRL (Brazilian Receivables)', linewidth=2)
plt.plot(data['date'], data['cmUSD']/100, label='cmUSD (USD Assets)', linewidth=2)
plt.plot(data['date'], data['cmBTC']/100, label='cmBTC (Bitcoin-backed)', linewidth=2)
plt.plot(data['date'], data['SELIC']/100, label='SELIC (Benchmark)', linestyle='--', alpha=0.7)

plt.title('Asset Yields Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Yield (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Quick Backtest: CCYOE Performance

In [ ]:
# Configure CCYOE
config = OptimizationConfig(
    under_supplied_allocation=0.40,  # 40% to under-supplied assets (cmBTC, cmUSD)
    strategic_growth_allocation=0.30,  # 30% to strategic growth
    proportional_allocation=0.20,     # 20% proportional to all holders
    treasury_allocation=0.10,         # 10% to protocol treasury
    rebalance_threshold=100,          # Rebalance when excess yield > 1%
    target_yields={
        'cmBTC': 500,   # 5% target
        'cmUSD': 1400,  # 14% target
        'cmBRL': 2000   # 20% target
    }
)

# Run backtest
backtester = CCYOEBacktester(data, config)
results = backtester.run_backtest('2023-01-01', '2024-01-01')

print("🎯 CCYOE Backtest Results:")
print(f"   Total Return: {results.total_return:.1%}")
print(f"   Sharpe Ratio: {results.sharpe_ratio:.2f}")
print(f"   Max Drawdown: {results.max_drawdown:.1%}")
print(f"   Rebalancing Events: {results.total_rebalances}")
print(f"   Average Excess Yield: {results.avg_excess_yield:.0f} bp")

print("\n✨ Yield Improvements:")
for asset, improvement in results.yield_improvement.items():
    print(f"   {asset}: +{improvement:.0f} bp boost")

## Performance Visualization

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(12, 6))
cumulative_returns = (1 + results.daily_returns).cumprod() - 1
plt.plot(results.daily_returns.index, cumulative_returns * 100, 
         linewidth=2, label='CCYOE Strategy')

# Add benchmark (SELIC)
selic_daily = data['SELIC'].pct_change().dropna()
if len(selic_daily) > 0:
    selic_cumulative = (1 + selic_daily).cumprod() - 1
    plt.plot(selic_cumulative.index, selic_cumulative * 100, 
             linestyle='--', alpha=0.7, label='SELIC Benchmark')

plt.title('CCYOE Strategy Performance', fontsize=14, fontweight='bold')
plt.ylabel('Cumulative Return (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Key metrics summary
print("\n📊 Key Insights:")
print(f"✅ The CCYOE system generated {results.avg_excess_yield:.0f} bp of excess yield")
print(f"✅ Achieved {results.sharpe_ratio:.2f} Sharpe ratio through yield optimization")
print(f"✅ Performed {results.total_rebalances} rebalancing operations")
print(f"✅ Enhanced yields across all assets through cross-collateral optimization")

## Next Steps

This was just a quick introduction! Explore the other notebooks for deeper analysis:

1. **02_data_analysis.ipynb** - Comprehensive data analysis and yield patterns
2. **03_backtesting.ipynb** - Detailed CCYOE backtesting and strategy evaluation
3. **04_optimization.ipynb** - Parameter optimization and sensitivity analysis
4. **05_risk_analysis.ipynb** - Risk assessment and stress testing

## Key Takeaways

- 🚀 CCYOE creates a **unified yield pool** that benefits all protocol users
- 📈 **Cross-collateral optimization** enhances yields beyond traditional DeFi
- ⚡ **Automated rebalancing** ensures optimal yield distribution
- 🛡️ **Risk isolation** maintains asset security while maximizing returns
- 🌍 **Network effects** mean success in one asset benefits the entire ecosystem